In [2]:
# %% [markdown]
# # Smart Agriculture Simulation: AI-Driven Crop Yield Prediction
# 
# This notebook simulates IoT sensor data and trains an LSTM model to predict crop yields.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

# %%
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# %% [markdown]
# ## 1. IoT Sensor Data Simulation

# %%
class AgricultureSensorSimulator:
    def __init__(self, start_date, days=180):
        self.start_date = start_date
        self.days = days
        self.dates = [start_date + timedelta(days=x) for x in range(days)]
        
    def generate_seasonal_pattern(self, base_value, amplitude, phase_shift=0):
        """Generate seasonal patterns using sine waves"""
        t = np.arange(self.days)
        seasonal = base_value + amplitude * np.sin(2 * np.pi * t / 90 + phase_shift)
        return seasonal
    
    def add_noise(self, data, noise_level):
        """Add realistic noise to sensor data"""
        noise = np.random.normal(0, noise_level, len(data))
        return data + noise
    
    def simulate_soil_moisture(self):
        """Simulate soil moisture data with irrigation events"""
        base_pattern = self.generate_seasonal_pattern(35, 15, phase_shift=0)
        
        # Add irrigation events (sudden increases followed by gradual decreases)
        moisture = base_pattern.copy()
        for i in range(0, self.days, 7):  # Weekly irrigation
            if i + 2 < self.days:
                moisture[i:i+2] += np.random.uniform(15, 25, 2)
        
        moisture = self.add_noise(moisture, 2)
        return np.clip(moisture, 10, 60)
    
    def simulate_temperature(self):
        """Simulate air temperature with diurnal and seasonal variations"""
        seasonal = self.generate_seasonal_pattern(25, 8, phase_shift=1.57)
        diurnal = 5 * np.sin(2 * np.pi * np.arange(self.days) * 4)  # 4 cycles per day
        temperature = seasonal + diurnal
        return self.add_noise(temperature, 1.5)
    
    def simulate_humidity(self):
        """Simulate relative humidity (inverse relationship with temperature)"""
        base_humidity = self.generate_seasonal_pattern(65, 20, phase_shift=3.14)
        return np.clip(self.add_noise(base_humidity, 3), 30, 95)
    
    def simulate_npk_levels(self):
        """Simulate Nitrogen, Phosphorus, Potassium levels"""
        # Initial levels with gradual depletion and fertilization events
        time = np.arange(self.days)
        
        # Nitrogen
        N_base = 25 - 0.1 * time
        for i in range(0, self.days, 30):  # Monthly fertilization  <-- CORRECTED
            if i < self.days:
                N_base[i] += np.random.uniform(8, 12)
        nitrogen = np.clip(self.add_noise(N_base, 1), 5, 40)
        
        # Phosphorus (more stable)
        P_base = 15 - 0.05 * time
        phosphorus = np.clip(self.add_noise(P_base, 0.5), 8, 25)
        
        # Potassium
        K_base = 20 - 0.08 * time
        potassium = np.clip(self.add_noise(K_base, 0.8), 10, 30)
        
        return nitrogen, phosphorus, potassium
    
    def simulate_rainfall(self):
        """Simulate rainfall events"""
        rainfall = np.zeros(self.days)
        # Random rainfall events
        for i in range(self.days):
            if np.random.random() < 0.15:  # 15% chance of rain each day
                rainfall[i] = np.random.exponential(5)
        
        # Add seasonal pattern (more rain in certain months)
        seasonal_effect = 0.5 + 0.5 * np.sin(2 * np.pi * np.arange(self.days) / 90 + 1.57)
        rainfall = rainfall * (1 + seasonal_effect)
        return rainfall
    
    def generate_dataset(self, num_seasons=5):
        """Generate multiple seasons of sensor data"""
        all_data = []
        
        for season in range(num_seasons):
            # Adjust base parameters slightly each season for variability
            season_factor = 0.9 + 0.2 * np.random.random()
            
            # Generate sensor data
            soil_moisture = self.simulate_soil_moisture() * season_factor
            temperature = self.simulate_temperature() * season_factor
            humidity = self.simulate_humidity()
            nitrogen, phosphorus, potassium = self.simulate_npk_levels()
            rainfall = self.simulate_rainfall()
            
            # Calculate NDVI (simulated plant health index)
            ndvi = 0.3 + 0.5 * (1 - np.exp(-0.03 * np.arange(self.days)))  # Logistic growth
            ndvi = self.add_noise(ndvi, 0.05)
            ndvi = np.clip(ndvi, 0.1, 0.9)
            
            # Create season dataframe
            season_dates = [self.start_date + timedelta(days=(x + season * self.days)) 
                           for x in range(self.days)]
            
            season_df = pd.DataFrame({
                'date': season_dates,
                'season': season,
                'day_in_season': np.arange(self.days),
                'soil_moisture': soil_moisture,
                'temperature': temperature,
                'humidity': humidity,
                'nitrogen': nitrogen,
                'phosphorus': phosphorus,
                'potassium': potassium,
                'rainfall': rainfall,
                'ndvi': ndvi
            })
            
            all_data.append(season_df)
        
        return pd.concat(all_data, ignore_index=True)

# %%
# Generate synthetic dataset
print("Generating synthetic agriculture sensor data...")
start_date = datetime(2020, 1, 1)
simulator = AgricultureSensorSimulator(start_date, days=120)  # 120-day growing seasons
df = simulator.generate_dataset(num_seasons=8)  # 8 seasons of data

print(f"Generated dataset shape: {df.shape}")
print("\nFirst few rows of data:")
df.head()

# %%
# Calculate yield based on sensor data (this will be our target variable)
def calculate_yield(season_data):
    """Calculate crop yield based on aggregated sensor readings"""
    # Ideal conditions multipliers
    moisture_score = np.mean(np.clip(season_data['soil_moisture'], 20, 40)) / 30
    temp_score = np.mean(np.clip(season_data['temperature'], 15, 30)) / 22.5
    nutrient_score = np.mean(season_data[['nitrogen', 'phosphorus', 'potassium']].mean(axis=1)) / 20
    ndvi_score = np.mean(season_data['ndvi'])
    
    # Combine scores with weights
    base_yield = 3.0  # tons/hectare base
    yield_multiplier = (0.3 * moisture_score + 0.3 * temp_score + 
                       0.2 * nutrient_score + 0.2 * ndvi_score)
    
    # Add some random variation
    random_variation = np.random.normal(1, 0.1)
    
    final_yield = base_yield * yield_multiplier * random_variation
    return np.clip(final_yield, 1.0, 6.0)

# Calculate yield for each season
yields = []
for season in df['season'].unique():
    season_data = df[df['season'] == season]
    yield_value = calculate_yield(season_data)
    yields.append({'season': season, 'yield_tons_hectare': yield_value})

yield_df = pd.DataFrame(yields)
print("Seasonal yields calculated:")
print(yield_df)

# %%
# Visualize sensor data for one season
plt.figure(figsize=(15, 12))

# Plot 1: Soil conditions
plt.subplot(3, 2, 1)
season_data = df[df['season'] == 0]
plt.plot(season_data['day_in_season'], season_data['soil_moisture'], label='Soil Moisture')
plt.plot(season_data['day_in_season'], season_data['temperature'], label='Temperature')
plt.xlabel('Days in Season')
plt.ylabel('Value')
plt.title('Soil Moisture and Temperature - Season 0')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Nutrient levels
plt.subplot(3, 2, 2)
plt.plot(season_data['day_in_season'], season_data['nitrogen'], label='Nitrogen')
plt.plot(season_data['day_in_season'], season_data['phosphorus'], label='Phosphorus')
plt.plot(season_data['day_in_season'], season_data['potassium'], label='Potassium')
plt.xlabel('Days in Season')
plt.ylabel('NPK Levels')
plt.title('Soil Nutrients - Season 0')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Environmental factors
plt.subplot(3, 2, 3)
plt.plot(season_data['day_in_season'], season_data['humidity'], label='Humidity', color='green')
plt.plot(season_data['day_in_season'], season_data['rainfall'], label='Rainfall', color='blue')
plt.xlabel('Days in Season')
plt.ylabel('Value')
plt.title('Humidity and Rainfall - Season 0')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Plant health (NDVI)
plt.subplot(3, 2, 4)
plt.plot(season_data['day_in_season'], season_data['ndvi'], label='NDVI', color='red')
plt.xlabel('Days in Season')
plt.ylabel('NDVI')
plt.title('Plant Health (NDVI) - Season 0')
plt.grid(True, alpha=0.3)

# Plot 5: Yield distribution
plt.subplot(3, 2, 5)
plt.bar(yield_df['season'], yield_df['yield_tons_hectare'], color='skyblue', alpha=0.7)
plt.xlabel('Season')
plt.ylabel('Yield (tons/hectare)')
plt.title('Crop Yield by Season')
plt.grid(True, alpha=0.3)

# Plot 6: Correlation heatmap
plt.subplot(3, 2, 6)
correlation_cols = ['soil_moisture', 'temperature', 'humidity', 'nitrogen', 
                   'phosphorus', 'potassium', 'rainfall', 'ndvi']
corr_matrix = df[correlation_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Sensor Data Correlation Matrix')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 2. Data Preprocessing for LSTM

# %%
def prepare_sequences(df, yield_df, sequence_length=30, test_seasons=2):
    """Prepare sequences for LSTM training"""
    
    # Features to use for prediction
    feature_columns = ['soil_moisture', 'temperature', 'humidity', 
                      'nitrogen', 'phosphorus', 'potassium', 'rainfall', 'ndvi']
    
    # Normalize features
    scaler = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[feature_columns] = scaler.fit_transform(df[feature_columns])
    
    X, y = [], []
    seasons = sorted(df['season'].unique())
    
    # Create sequences for each season
    for season in seasons:
        if season not in yield_df['season'].values:
            continue
            
        season_data = df_scaled[df_scaled['season'] == season]
        season_yield = yield_df[yield_df['season'] == season]['yield_tons_hectare'].values[0]
        
        # Create sequences of specified length
        for i in range(len(season_data) - sequence_length + 1):
            sequence = season_data[feature_columns].iloc[i:i+sequence_length].values
            X.append(sequence)
            y.append(season_yield)
    
    X = np.array(X)
    y = np.array(y)
    
    # Split into train/test based on seasons
    train_seasons = seasons[:-test_seasons]
    test_seasons = seasons[-test_seasons:]
    
    train_mask = [s in train_seasons for s in df['season']]
    test_mask = [s in test_seasons for s in df['season']]
    
    X_train = X[train_mask[:len(X)]]
    X_test = X[test_mask[:len(X)]]
    y_train = y[train_mask[:len(y)]]
    y_test = y[test_mask[:len(y)]]
    
    print(f"Training set: {X_train.shape[0]} sequences")
    print(f"Test set: {X_test.shape[0]} sequences")
    print(f"Sequence shape: {X_train.shape[1:]")
    
    return X_train, X_test, y_train, y_test, scaler, feature_columns

# %%
# Prepare data for LSTM
SEQUENCE_LENGTH = 30  # Use 30 days of data to predict yield
TEST_SEASONS = 2      # Use last 2 seasons for testing

X_train, X_test, y_train, y_test, scaler, feature_columns = prepare_sequences(
    df, yield_df, SEQUENCE_LENGTH, TEST_SEASONS
)

# %% [markdown]
# ## 3. LSTM Model for Yield Prediction

# %%
def create_lstm_model(input_shape, lstm_units=64, dropout_rate=0.3):
    """Create LSTM model for yield prediction"""
    
    model = Sequential([
        # First LSTM layer
        LSTM(lstm_units, return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        
        # Second LSTM layer
        LSTM(lstm_units // 2, return_sequences=False),
        Dropout(dropout_rate),
        
        # Dense layers
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(16, activation='relu'),
        
        # Output layer (single value for yield prediction)
        Dense(1, activation='linear')  # Linear activation for regression
    ])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# %%
# Create and display model
input_shape = (X_train.shape[1], X_train.shape[2])
model = create_lstm_model(input_shape)
model.summary()

# %%
# Train the model
print("Training LSTM model...")

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7
        )
    ]
)

# %% [markdown]
# ## 4. Model Evaluation

# %%
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %%
# Make predictions
y_pred = model.predict(X_test).flatten()

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Model Performance Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.4f} tons/hectare")
print(f"Root Mean Square Error (RMSE): {rmse:.4f} tons/hectare")
print(f"R² Score: {r2:.4f}")

# %%
# Visualize predictions vs actual
plt.figure(figsize=(12, 5))

# Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.7, s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Yield (tons/hectare)')
plt.ylabel('Predicted Yield (tons/hectare)')
plt.title(f'Actual vs Predicted Yield\nR² = {r2:.4f}')
plt.grid(True, alpha=0.3)

# Residual plot
plt.subplot(1, 2, 2)
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Yield')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %%
# Feature importance analysis (using permutation importance simulation)
def calculate_feature_importance(model, X_test, y_test, feature_names):
    """Calculate approximate feature importance using permutation"""
    baseline_score = model.evaluate(X_test, y_test, verbose=0)[1]  # MAE
    importance_scores = []
    
    for i in range(X_test.shape[2]):
        X_permuted = X_test.copy()
        # Shuffle the feature
        X_permuted[:, :, i] = np.random.permutation(X_permuted[:, :, i])
        permuted_score = model.evaluate(X_permuted, y_test, verbose=0)[1]
        
        # Importance is the decrease in performance
        importance = baseline_score - permuted_score
        importance_scores.append(importance)
    
    return pd.DataFrame({
        'feature': feature_names,
        'importance': importance_scores
    }).sort_values('importance', ascending=True)

# Calculate feature importance
feature_importance = calculate_feature_importance(model, X_test, y_test, feature_columns)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance (Decrease in MAE when permuted)')
plt.title('Feature Importance for Yield Prediction')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 5. Real-time Prediction Simulation

# %%
def simulate_real_time_prediction(model, df, scaler, feature_columns, season=6, sequence_length=30):
    """Simulate real-time yield prediction during a growing season"""
    
    season_data = df[df['season'] == season].copy()
    season_data[feature_columns] = scaler.transform(season_data[feature_columns])
    
    predictions = []
    confidence_intervals = []
    
    # Make predictions at different points in the season
    for day in range(sequence_length, len(season_data) + 1, 7):  # Weekly predictions
        sequence = season_data[feature_columns].iloc[day-sequence_length:day].values
        sequence = sequence.reshape(1, sequence_length, len(feature_columns))
        
        prediction = model.predict(sequence, verbose=0)[0][0]
        predictions.append((day, prediction))
        
        # Simple confidence interval based on day of prediction
        confidence = min(0.95, 0.3 + (day / len(season_data)) * 0.65)
        confidence_intervals.append(confidence)
    
    return predictions, confidence_intervals

# %%
# Run real-time simulation
predictions, confidences = simulate_real_time_prediction(
    model, df, scaler, feature_columns, season=6
)

# Get actual yield for comparison
actual_yield = yield_df[yield_df['season'] == 6]['yield_tons_hectare'].values[0]

# Plot real-time prediction progression
days, preds = zip(*predictions)

plt.figure(figsize=(12, 6))
plt.plot(days, preds, 'bo-', label='Predicted Yield', linewidth=2, markersize=6)
plt.axhline(y=actual_yield, color='r', linestyle='--', linewidth=2, label=f'Actual Yield: {actual_yield:.2f} t/ha')

# Add confidence intervals
for i, (day, pred, conf) in enumerate(zip(days, preds, confidences)):
    error_margin = (1 - conf) * 0.5  # Error margin decreases as confidence increases
    plt.fill_between([day-2, day+2], 
                    [pred * (1 - error_margin), pred * (1 - error_margin)],
                    [pred * (1 + error_margin), pred * (1 + error_margin)],
                    alpha=0.2, color='blue')

plt.xlabel('Day in Season')
plt.ylabel('Predicted Yield (tons/hectare)')
plt.title('Real-time Yield Prediction Progression\n(Simulation for Season 6)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(2, 5)  # Fixed y-axis for better comparison
plt.show()

# %%
# Final model performance summary
print("="*50)
print("SMART AGRICULTURE YIELD PREDICTION - FINAL REPORT")
print("="*50)
print(f"Model: LSTM with {model.count_params():,} parameters")
print(f"Training seasons: {len(df['season'].unique()) - TEST_SEASONS}")
print(f"Test seasons: {TEST_SEASONS}")
print(f"Sequence length: {SEQUENCE_LENGTH} days")
print(f"Features used: {len(feature_columns)} sensor measurements")
print("\nPerformance Metrics:")
print(f"MAE:  {mae:.3f} tons/hectare")
print(f"RMSE: {rmse:.3f} tons/hectare")
print(f"R²:   {r2:.3f}")
print("\nMost Important Features:")
print(feature_importance.tail(3)[['feature', 'importance']].to_string(index=False))

# %%
# Save the model and scaler for future use
model.save('crop_yield_lstm_model.h5')

import joblib
joblib.dump(scaler, 'feature_scaler.pkl')

print("\nModel and scaler saved successfully!")
print("Files saved: 'crop_yield_lstm_model.h5' and 'feature_scaler.pkl'")

SyntaxError: f-string: expecting '}' (2835068495.py, line 308)